# 导入工具包

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from gensim.models import Word2Vec
import lightgbm as lgb

/root/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# 读取数据

In [2]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
results = pd.read_csv("../data/results.csv")

# 查看训练数据

In [3]:
train.head()

,id,level_1,level_2,level_3,level_4,content,label
0,0,工业/危化品类（现场）—2016版,（二）电气安全,6、移动用电产品、电动工具及照明,1、移动使用的用电产品和I类电动工具的绝缘线，必须采用三芯(单相)或四芯(三相)多股铜芯橡套软线。,"使用移动手动电动工具,外接线绝缘皮破损,应停止使用.",0
1,1,工业/危化品类（现场）—2016版,（一）消防检查,1、防火巡查,3、消防设施、器材和消防安全标志是否在位、完整；,一般,1
2,2,工业/危化品类（现场）—2016版,（一）消防检查,2、防火检查,6、重点工种人员以及其他员工消防知识的掌握情况；,消防知识要加强,0
3,3,工业/危化品类（现场）—2016版,（一）消防检查,1、防火巡查,3、消防设施、器材和消防安全标志是否在位、完整；,消防通道有货物摆放 清理不及时,0
4,4,工业/危化品类（现场）—2016版,（一）消防检查,1、防火巡查,4、常闭式防火门是否处于关闭状态，防火卷帘下是否堆放物品影响使用；,防火门打开状态,0


# 查看训练数据条数

In [4]:
train.shape

(12000, 7)

# 查看测试数据

In [5]:
test.head()

,id,level_1,level_2,level_3,level_4,content
0,0,交通运输类（现场）—2016版,（一）消防安全,2、防火检查,2、安全疏散通道、疏散指示标志、应急照明和安全出口情况。,RB1洗地机占用堵塞安全通道
1,1,工业/危化品类（选项）—2016版,（二）仓库,1、一般要求,1、库房内储存物品应分类、分堆、限额存放。,未分类堆放
2,2,工业/危化品类（现场）—2016版,（一）消防检查,1、防火巡查,3、消防设施、器材和消防安全标志是否在位、完整；,消防设施、器材和消防安全标志是否在位、完整
3,3,商贸服务教文卫类（现场）—2016版,（二）电气安全,3、电气线路及电源插头插座,3、电源插座、电源插头应按规定正确接线。,插座随意放在电器旁边
4,4,商贸服务教文卫类（现场）—2016版,（一）消防检查,1、防火巡查,6、其他消防安全情况。,检查中发现一瓶灭火器过期


# 训练集去重

In [6]:
train = train.drop_duplicates(['level_1', 'level_2', 'level_3', 'level_4', 'content', 'label'])

# 查看去重训练数据条数

In [7]:
train.shape

(10619, 7)

In [8]:
# !pip install textvec

# 构建特征 使用word2vec

In [9]:
train['text'] = ( train['content']
              ).map(lambda x:' '.join(list(str(x))))
test['text'] = (test['content']
             ).map(lambda x:' '.join(list(str(x))))

In [10]:
import jieba

In [11]:
# 切词

In [12]:
train['text'] = ( train['content']
              ).map(lambda x:' '.join(jieba.cut(str(x))))
test['text'] = (test['content']
             ).map(lambda x:' '.join(jieba.cut(str(x))))

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.749 seconds.
Prefix dict has been built successfully.


In [13]:
train['text'].head()

0    使用 移动 手动 电动工具 , 外 接线 绝缘 皮 破损 , 应 停止使用 .
1                                         一般
2                                 消防 知识 要 加强
3                     消防通道 有 货物 摆放   清理 不 及时
4                                  防火门 打开 状态
Name: text, dtype: object

In [14]:
import gensim

In [15]:
gensim.models.word2vec?

In [16]:
help(gensim.models.word2vec)

Help on module gensim.models.word2vec in gensim.models:

NAME
    gensim.models.word2vec

DESCRIPTION
    Introduction
    
    This module implements the word2vec family of algorithms, using highly optimized C routines,
    data streaming and Pythonic interfaces.
    
    The word2vec algorithms include skip-gram and CBOW models, using either
    hierarchical softmax or negative sampling: `Tomas Mikolov et al: Efficient Estimation of Word Representations
    in Vector Space <https://arxiv.org/pdf/1301.3781.pdf>`_, `Tomas Mikolov et al: Distributed Representations of Words
    and Phrases and their Compositionality <https://arxiv.org/abs/1310.4546>`_.
    
    Other embeddings
    
    There are more ways to train word vectors in Gensim than just Word2Vec.
    See also :class:`~gensim.models.doc2vec.Doc2Vec`, :class:`~gensim.models.fasttext.FastText`.
    
    The training algorithms were originally ported from the C package https://code.google.com/p/word2vec/
    and extended with add

In [19]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
path = get_tmpfile("word2vec.model")
model = Word2Vec(common_texts, size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")

In [20]:
from gensim.models import Word2Vec

In [22]:
model = Word2Vec(train['text'], size=100, window=5, min_count=1, workers=4)

In [23]:
model.wv.index2word

AttributeError: The index2word attribute has been replaced by index_to_key since Gensim 4.0.0.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4

In [43]:
model.wv['业']

array([ 0.74876255, -1.523528  , -0.50259686,  0.1642955 ,  0.66953623,
       -0.07708137, -0.24506499,  0.11948275,  0.18651788, -0.3547346 ,
        0.6613543 ,  0.8570319 ,  1.4272025 , -0.7080727 , -0.11836702,
       -0.39037162, -0.58911514,  1.4818192 ,  0.25988027, -1.5528195 ,
       -0.4836867 ,  0.9258825 ,  0.3224472 , -0.9981493 ,  0.22589695,
       -0.5994131 , -0.5883326 , -0.9485425 ,  0.53912103, -0.8944482 ,
        0.01652133,  0.37102234, -0.5000812 , -0.32099706, -0.04379452,
        0.07765715, -0.3867584 , -0.17166753,  0.45220587, -0.16407226,
       -0.3244585 ,  1.9656328 ,  0.7018858 ,  0.16246897, -0.1254905 ,
       -0.34526324,  1.3757665 ,  0.4181869 ,  0.16025238, -0.13505958,
        0.5609357 , -0.56489   ,  0.5345593 , -0.28472492, -0.6219791 ,
       -0.02456244,  0.07932727,  0.5837283 , -0.16772443,  1.3150066 ,
        1.0884486 ,  0.09345187,  0.54304105, -0.8637025 ,  0.07659232,
       -0.24300766, -0.20176442, -0.15016618, -0.03549685,  0.44

In [44]:
model.wv['业'].shape

(100,)

In [47]:
# skip model
model_skip = Word2Vec(train['text'], size=100, window=5, min_count=1, workers=4, sg=1)

In [31]:
train['text'] = ( train['content']
              ).map(lambda x:list(jieba.cut(str(x))))
test['text'] = (test['content']
             ).map(lambda x:list(jieba.cut(str(x))))

In [32]:
train['text'].head()

0    [使用, 移动, 手动, 电动工具, ,, 外, 接线, 绝缘, 皮, 破损, ,, 应, ...
1                                                 [一般]
2                                      [消防, 知识, 要, 加强]
3                      [消防通道, 有, 货物, 摆放,  , 清理, 不, 及时]
4                                        [防火门, 打开, 状态]
Name: text, dtype: object

In [39]:
train['text'].values.tolist()

[['使用',
  '移动',
  '手动',
  '电动工具',
  ',',
  '外',
  '接线',
  '绝缘',
  '皮',
  '破损',
  ',',
  '应',
  '停止使用',
  '.'],
 ['一般'],
 ['消防', '知识', '要', '加强'],
 ['消防通道', '有', '货物', '摆放', ' ', '清理', '不', '及时'],
 ['防火门', '打开', '状态'],
 ['防爆', '柜', '里面', '稀释剂', '，', '机油', '费', '混装'],
 ['已经', '整改'],
 ['逃生', '通道', '有', '货物', '阻挡', '。'],
 ['已', '整改'],
 ['通道', '黄色', '线标', '脱落', '，', '已', '及时', '重新', '标好', '线标', '。'],
 ['气压', '不够', '，', '已', '更换'],
 ['通道', '有', '堆放', '货物'],
 ['灭火器', '没', '检查', '，', '已', '检查', '，', '已', '整改'],
 ['三楼', '车间', '天花板', '脱落', '，', '找人', '维修'],
 ['车间', '一', '推台', '锯', '未', '安装', '防护罩'],
 ['有', '一处', '安全', '出口', '不', '畅通'],
 ['配电室', '长度', '超过', '7m', '时', '，', '没有', '设有', '两个', '出口'],
 ['疏散', '通道', '堆满', '货物'],
 ['未', '进行', '消防', '演练'],
 ['新', '员工', '不能', '熟练掌握', '消防安全', '知识', '，', '已', '培训', '教育'],
 ['走廊', '通道', '有', '一个', '灭火器', '到期', '，', '立即', '整改'],
 ['巡查', '中', '发现', '紧急', '疏散', '指示灯', '故障', '，', '已经', '更换'],
 ['疏散', '指示灯', '故障'],
 ['安全', '疏散', '标志', '没有'],
 ['未',
  '组织',
  '员工

In [40]:
model_word = Word2Vec(train['text'].values.tolist(), size=100, window=5, min_count=1, workers=4)

In [42]:
model_word.wv.index2word

['，',
 '。',
 '安全',
 '灭火器',
 '有',
 '已',
 '通道',
 '未',
 '整改',
 ' ',
 '标志',
 '的',
 '更换',
 '出口',
 '没有',
 '杂物',
 '堆放',
 '货物',
 '立即',
 '疏散',
 '车间',
 '不',
 '、',
 '堵塞',
 '配电箱',
 '消防通道',
 '了',
 '及时',
 '应急',
 '检查',
 '损坏',
 '警示',
 '被',
 '无',
 '指示',
 '脱落',
 '消防',
 '摆放',
 '已经',
 '培训',
 '新',
 '一个',
 '在',
 '安装',
 '清理',
 '进行',
 '发现',
 '防火门',
 '记录',
 '应急灯',
 '张贴',
 '标识',
 '照明灯',
 '员工',
 '放置',
 '上',
 '仓库',
 '过期',
 '物品',
 '开关',
 '1',
 '部分',
 '使用',
 '消防栓',
 '插座',
 '2',
 '要求',
 '好',
 '临时',
 '作业',
 '；',
 '坏',
 '生产',
 '重新',
 '现已',
 '灯不亮',
 '畅通',
 '“',
 '当心',
 '消防安全',
 '二楼',
 '和',
 '触电',
 '一楼',
 '”',
 '一处',
 '楼',
 '并',
 '演练',
 '人员',
 '指示灯',
 '后',
 '现场',
 '占用',
 '巡查',
 '教育',
 '不足',
 '需',
 '关闭',
 '没',
 '电线',
 '应急照明',
 '.',
 '防火',
 '隐患',
 '！',
 '电箱',
 '破损',
 '点检',
 '安排',
 '位置',
 '消防器材',
 '一',
 '故障',
 '内',
 '个别',
 '定期',
 '栋',
 '不够',
 '情况',
 '门',
 '失效',
 '电源插座',
 '放',
 '漏电',
 '器材',
 '线',
 '缺失',
 '设置',
 '标示',
 '存在',
 '门口',
 '压力',
 '指示牌',
 '接地线',
 ',',
 '及',
 '设备',
 '需要',
 '更新',
 '遮挡',
 '个',
 '完毕',
 '3',
 '防护罩',
 '灭火

In [45]:
model_word.wv['仓库']

array([ 0.11700831,  0.17178163,  0.7863628 , -0.21773534, -0.27272156,
       -0.35511106, -0.6329898 , -0.26672664, -0.31122753,  0.57652026,
        0.21903372, -0.52168536,  0.08532668, -1.1270018 ,  0.18241176,
       -0.5040042 ,  0.5993073 , -0.24973182, -0.55129576, -0.26039174,
       -0.53371197, -0.12542738,  0.62626135,  0.11287792, -0.8340116 ,
        0.08784495,  0.62378   , -0.53252345, -0.36973277,  0.52708465,
       -0.19629198, -0.39003348,  0.89453316,  0.002386  ,  0.15012625,
        0.604828  , -1.3763983 ,  0.2665835 ,  0.21268262,  0.08306839,
        0.08126837, -0.19527033, -1.014167  , -0.18946478, -0.01643675,
       -0.5518825 ,  0.28129882, -0.39331937, -0.15660612, -0.21991937,
        0.74528646,  0.3067103 ,  0.19970089, -0.44701633, -0.826218  ,
       -0.37129968,  0.7360679 , -0.09438144, -0.08160633,  0.6419061 ,
       -0.35647723,  0.37023443, -0.67849016,  0.03259431, -0.51178217,
        0.448171  ,  0.8399702 , -0.29937932,  0.233349  ,  0.43

In [46]:
Word2Vec?

In [68]:
def get_vec(word_list, model):
    init = np.array([0.0]*100)
    index = 0
    for word in word_list:
        if word in model.wv:  
            init += np.array(model.wv[word])
            index += 1
    if index == 0:
        return init
    return list(init / index)

In [62]:
get_vec(train['text'].values[0], model_word)

array([ 0.06325182,  0.0269619 ,  0.35827692, -0.11235228, -0.20698204,
       -0.23006965, -0.33353246, -0.06356141, -0.15537973,  0.29171457,
        0.06373887, -0.3304247 ,  0.03052058, -0.65059295,  0.0633381 ,
       -0.24924335,  0.31237718, -0.15429361, -0.36298618, -0.14653165,
       -0.17607548, -0.07022922,  0.35951458,  0.11678855, -0.48483831,
       -0.01613001,  0.34711947, -0.2378797 , -0.25884159,  0.32509941,
       -0.14192978, -0.17930104,  0.59957242,  0.00520837,  0.05434635,
        0.2529241 , -0.82836112,  0.11387875,  0.13923346,  0.03228319,
        0.05736054, -0.16239438, -0.49762101, -0.08512611, -0.03902122,
       -0.33012109,  0.15800153, -0.22128158, -0.16257685, -0.11625686,
        0.4448014 ,  0.19855433,  0.19338341, -0.2120464 , -0.3756567 ,
       -0.2323266 ,  0.40377687, -0.08570144, -0.04844976,  0.33246046,
       -0.30772991,  0.20803204, -0.43446559,  0.09732723, -0.36825712,
        0.28660449,  0.50729774, -0.15704825,  0.21213905,  0.26

In [63]:
# train['text'].values[0]

# 获取向量 直接求平均

In [69]:
train['vec'] = train['text'].map(lambda x: get_vec(x, model_word))

In [72]:
test['vec'] = test['text'].map(lambda x: get_vec(x, model_word))

In [73]:
train_X = np.array(train['vec'].values.tolist())
test_X = np.array(test['vec'].values.tolist())

In [74]:
test_X.shape

(18000, 100)

In [75]:
train_X.shape

(10619, 100)

In [76]:
train_y = train['label'].astype(int).values

In [17]:
# vectorizer.get_feature_names()[-10:]

# 五折交叉验证

In [77]:
params = {
        'task':'train',
        'boosting_type':'gbdt',
        'num_leaves': 31,
        'objective': 'binary', 
        'learning_rate': 0.05, 
        'bagging_freq': 2, 
        'max_bin':256,
        'num_threads': 32,
#         'metric':['binary_logloss','binary_error']
    } 

In [78]:
skf = StratifiedKFold(n_splits=5)

In [79]:
for index,(train_index, test_index) in enumerate(skf.split(train_X, train_y)):
    X_train, X_test = train_X[train_index], train_X[test_index]
    y_train, y_test = train_y[train_index], train_y[test_index]
    
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
    
    gbm = lgb.train(params,
                lgb_train,
                num_boost_round=1000,
                valid_sets=lgb_eval,
                early_stopping_rounds=10)
    y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
    pred = gbm.predict(test_X, num_iteration=gbm.best_iteration)
    if index == 0:
        pred_y_check, true_y_check = list(y_pred), list(y_test)
        pred_out=pred
    else:
        pred_y_check += list(y_pred)
        true_y_check += list(y_test)
        pred_out += pred
        

[1]	valid_0's binary_logloss: 0.243483
Training until validation scores don't improve for 10 rounds
[2]	valid_0's binary_logloss: 0.234249
[3]	valid_0's binary_logloss: 0.227049
[4]	valid_0's binary_logloss: 0.221133
[5]	valid_0's binary_logloss: 0.21582
[6]	valid_0's binary_logloss: 0.211375
[7]	valid_0's binary_logloss: 0.206936
[8]	valid_0's binary_logloss: 0.203704
[9]	valid_0's binary_logloss: 0.200724
[10]	valid_0's binary_logloss: 0.197262
[11]	valid_0's binary_logloss: 0.194711
[12]	valid_0's binary_logloss: 0.192477
[13]	valid_0's binary_logloss: 0.189993
[14]	valid_0's binary_logloss: 0.187784
[15]	valid_0's binary_logloss: 0.186055
[16]	valid_0's binary_logloss: 0.184411
[17]	valid_0's binary_logloss: 0.182289
[18]	valid_0's binary_logloss: 0.180749
[19]	valid_0's binary_logloss: 0.179077
[20]	valid_0's binary_logloss: 0.177758
[21]	valid_0's binary_logloss: 0.176642
[22]	valid_0's binary_logloss: 0.175484
[23]	valid_0's binary_logloss: 0.174993
[24]	valid_0's binary_logloss

[48]	valid_0's binary_logloss: 0.173875
[49]	valid_0's binary_logloss: 0.173627
[50]	valid_0's binary_logloss: 0.173344
[51]	valid_0's binary_logloss: 0.173128
[52]	valid_0's binary_logloss: 0.172906
[53]	valid_0's binary_logloss: 0.172564
[54]	valid_0's binary_logloss: 0.172258
[55]	valid_0's binary_logloss: 0.171957
[56]	valid_0's binary_logloss: 0.171893
[57]	valid_0's binary_logloss: 0.171607
[58]	valid_0's binary_logloss: 0.171239
[59]	valid_0's binary_logloss: 0.170937
[60]	valid_0's binary_logloss: 0.17066
[61]	valid_0's binary_logloss: 0.170213
[62]	valid_0's binary_logloss: 0.170049
[63]	valid_0's binary_logloss: 0.169884
[64]	valid_0's binary_logloss: 0.170005
[65]	valid_0's binary_logloss: 0.169724
[66]	valid_0's binary_logloss: 0.169539
[67]	valid_0's binary_logloss: 0.169418
[68]	valid_0's binary_logloss: 0.169071
[69]	valid_0's binary_logloss: 0.168678
[70]	valid_0's binary_logloss: 0.168801
[71]	valid_0's binary_logloss: 0.168678
[72]	valid_0's binary_logloss: 0.168372
[

[31]	valid_0's binary_logloss: 0.18316
[32]	valid_0's binary_logloss: 0.18271
[33]	valid_0's binary_logloss: 0.181966
[34]	valid_0's binary_logloss: 0.181327
[35]	valid_0's binary_logloss: 0.180636
[36]	valid_0's binary_logloss: 0.180308
[37]	valid_0's binary_logloss: 0.180035
[38]	valid_0's binary_logloss: 0.179645
[39]	valid_0's binary_logloss: 0.179254
[40]	valid_0's binary_logloss: 0.178932
[41]	valid_0's binary_logloss: 0.178695
[42]	valid_0's binary_logloss: 0.178208
[43]	valid_0's binary_logloss: 0.177752
[44]	valid_0's binary_logloss: 0.177251
[45]	valid_0's binary_logloss: 0.176924
[46]	valid_0's binary_logloss: 0.176515
[47]	valid_0's binary_logloss: 0.176526
[48]	valid_0's binary_logloss: 0.176253
[49]	valid_0's binary_logloss: 0.175622
[50]	valid_0's binary_logloss: 0.175556
[51]	valid_0's binary_logloss: 0.175333
[52]	valid_0's binary_logloss: 0.175275
[53]	valid_0's binary_logloss: 0.175238
[54]	valid_0's binary_logloss: 0.174968
[55]	valid_0's binary_logloss: 0.174639
[5

# 线下验证

In [80]:
for i in range(10):
    pred = [int(x) for x in np.where(np.array(pred_y_check) >= i/10.0,1,0)]
    scores = f1_score(true_y_check,pred)
    print(i/10.0, scores)

0.0 0.13423526311165773
0.1 0.5178669598680593
0.2 0.5967503692762186
0.3 0.5831960461285007
0.4 0.5686956521739132
0.5 0.5411334552102376
0.6 0.527306967984934
0.7 0.5053037608486016
0.8 0.458
0.9 0.36012861736334406


In [224]:
for i in range(10):
    pred = [int(x) for x in np.where(np.array(pred_y_check) >= i/10.0,1,0)]
    scores = f1_score(true_y_check,pred)
    print(i, scores)

0 0.13423526311165773
1 0.7660032275416891
2 0.8048192771084337
3 0.8115015974440896
4 0.8158602150537635
5 0.8053408292340126
6 0.7817109144542772
7 0.7478532396565183
8 0.7043407043407044
9 0.611659192825112
